In [0]:
import pandas as pd
income = pd.read_csv("Unemployment.csv")
income = income [['FIPStxt','Stabr','area_name', 'Unemployment_rate_2019', 'Median_Household_Income_2018']]

In [0]:
state_val = income[income['FIPStxt']%1000 ==0]
county_val = income[~(income['FIPStxt']%1000 ==0)]
county_val

In [0]:
election_16 = pd.read_csv("2016_US_County_Level_Presidential_Results.csv")

election_16



In [12]:
gop = election_16[~(election_16['per_dem']>election_16['per_gop'])]
gop['Government'] = 'GOP'

dem = election_16[election_16['per_dem']>election_16['per_gop']]
dem['Government'] = 'DEM'
election_result16 =pd.concat([gop, dem], ignore_index=True)

election_result16 = election_result16[['Government','combined_fips']].sort_values(by ='combined_fips')

election_result16

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Government,combined_fips
29,GOP,1001
30,GOP,1003
31,GOP,1005
32,GOP,1007
33,GOP,1009
...,...,...
2650,GOP,56037
3140,DEM,56039
2651,GOP,56041
2652,GOP,56043


In [0]:
county_val = county_val.merge(election_result16,'inner',left_on='FIPStxt',right_on='combined_fips').drop(columns =['combined_fips'])

In [31]:
population = pd.read_csv("PopulationEstimates.csv",encoding="ISO-8859-1")
population = population [['FIPStxt','POP_ESTIMATE_2019']]
county_pop = population[~(population['FIPStxt']%1000 ==0)]
county_pop

,FIPStxt,POP_ESTIMATE_2019
2,1001,"55,869"
3,1003,"223,234"
4,1005,"24,686"
5,1007,"22,394"
6,1009,"57,826"
...,...,...
3268,72145,"50,023"
3269,72147,"8,386"
3270,72149,"21,372"
3271,72151,"32,282"


In [0]:
county_val = county_val.merge(county_pop,'inner','FIPStxt')

In [33]:
county_area =pd.read_csv("/content/drive/My Drive/Grad Courses/CSE547/COVID19 data analysis/demographics/census2016/safegraph_open_census_data/metadata/cbg_geographic_data.csv")
county_area.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220333 entries, 0 to 220332
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   census_block_group  220333 non-null  int64  
 1   amount_land         220333 non-null  float64
 2   amount_water        220333 non-null  int64  
 3   latitude            220333 non-null  float64
 4   longitude           220333 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 8.4 MB


In [0]:
l =[]
for val in county_area.census_block_group:
  arr = [int(i) for i in str(val)]
  #  print(val)
  if len(arr)<12:
    arr.insert(0,0)
  state = arr[0]*10+arr[1]
  s_c = state *1000 + arr[2]*100+arr[3]*10+arr[4]
  l.append(s_c)
len(l)
county_area['FIPStxt'] = l

county_land = county_area[['FIPStxt','amount_land']]


In [0]:
county_land = county_land.groupby(by = 'FIPStxt').mean()

In [0]:
county_location = county_area[['FIPStxt','latitude','longitude']]
county_location = county_location.groupby(by = 'FIPStxt').mean()

In [0]:
county_val = county_val.merge(county_land,'inner','FIPStxt')

In [0]:
county_val = county_val.merge(county_location,'inner','FIPStxt')

In [0]:
county_val.to_csv('county_demographics.csv')